In [1]:
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import CategoricalCrossentropy

In [2]:
def load_X_file():
    npy_file = "data/X.npy"
    df_file = "X_df.txt"
        
    if not os.path.exists(df_file):
        npy = np.load(npy_file)
        df = pd.DataFrame([x.flatten() for x in npy])
        df_file_id = open(df_file,'wb')
        pickle.dump(df,df_file_id)
        df_file_id.close()
    else:
        with open(df_file, "rb") as fid:
            df = pickle.load(fid,encoding="bytes")

    return df
    

In [3]:
X_df = load_X_file()

In [4]:
X_df

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,75,78,80,74,76,78,83,84,83,101,...,83,101,102,94,98,101,90,90,97,89
1,69,73,73,75,78,78,75,77,77,77,...,73,78,78,79,103,103,100,101,104,96
2,72,72,72,78,78,78,78,78,74,114,...,74,71,73,72,82,83,80,95,97,91
3,68,67,67,75,72,73,89,81,81,141,...,79,68,69,72,76,73,74,95,87,87
4,73,73,75,74,73,75,75,72,70,111,...,99,81,78,87,66,64,68,71,71,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39204,12,11,12,11,11,13,13,11,13,13,...,13,12,11,13,10,10,14,11,11,12
39205,11,9,10,10,9,11,10,9,11,11,...,11,10,9,12,11,10,13,11,11,13
39206,10,10,12,12,12,13,13,11,13,11,...,10,10,10,12,11,11,13,11,11,13
39207,13,12,15,13,13,16,13,12,13,10,...,17,13,12,14,11,9,11,11,10,11


In [5]:
scaler = StandardScaler()
X_df = scaler.fit_transform(X_df) 


In [6]:
y = np.load("data/y.npy")
y.shape

(39209, 43)

In [7]:
y_df = pd.DataFrame(y)
y_df.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.1)
X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(X_train_df, y_train_df, test_size=0.1)

In [9]:
nn = Sequential()

nn.add(Dense(1800, input_dim=3072, activation='relu'))
nn.add(Dense(900, activation='relu'))
nn.add(Dense(450, activation='relu'))
nn.add(Dense(43, activation='softmax'))
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1800)              5531400   
_________________________________________________________________
dense_1 (Dense)              (None, 900)               1620900   
_________________________________________________________________
dense_2 (Dense)              (None, 450)               405450    
_________________________________________________________________
dense_3 (Dense)              (None, 43)                19393     
Total params: 7,577,143
Trainable params: 7,577,143
Non-trainable params: 0
_________________________________________________________________


In [10]:
# compile model
nn.compile(loss=CategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])

In [11]:
model = nn.fit(X_train_df, y_train_df, validation_data=(X_val_df, y_val_df), epochs=100, batch_size=64)

Epoch 1/100
497/497 [==============================] - 20s 41ms/step - loss: 1.6099 - accuracy: 0.5703 - val_loss: 0.8235 - val_accuracy: 0.7719
Epoch 2/100
497/497 [==============================] - 20s 41ms/step - loss: 0.6477 - accuracy: 0.8054 - val_loss: 0.9044 - val_accuracy: 0.7968
Epoch 3/100
497/497 [==============================] - 21s 42ms/step - loss: 0.5083 - accuracy: 0.8479 - val_loss: 0.4663 - val_accuracy: 0.8671
Epoch 4/100
497/497 [==============================] - 21s 42ms/step - loss: 0.4190 - accuracy: 0.8721 - val_loss: 0.4588 - val_accuracy: 0.8728
Epoch 5/100
497/497 [==============================] - 21s 43ms/step - loss: 0.3687 - accuracy: 0.8878 - val_loss: 0.6885 - val_accuracy: 0.8138
Epoch 6/100
497/497 [==============================] - 21s 42ms/step - loss: 0.3368 - accuracy: 0.8967 - val_loss: 0.6594 - val_accuracy: 0.8719
Epoch 7/100
497/497 [==============================] - 22s 44ms/step - loss: 0.3500 - accuracy: 0.9003 - val_loss: 0.4003 - val_ac

497/497 [==============================] - 21s 42ms/step - loss: 0.1741 - accuracy: 0.9573 - val_loss: 0.4277 - val_accuracy: 0.9470
Epoch 58/100
497/497 [==============================] - 21s 41ms/step - loss: 0.1881 - accuracy: 0.9570 - val_loss: 0.5657 - val_accuracy: 0.9416
Epoch 59/100
497/497 [==============================] - 21s 42ms/step - loss: 0.1985 - accuracy: 0.9578 - val_loss: 0.4851 - val_accuracy: 0.9456
Epoch 60/100
497/497 [==============================] - 21s 42ms/step - loss: 0.2465 - accuracy: 0.9496 - val_loss: 0.5956 - val_accuracy: 0.9150
Epoch 61/100
497/497 [==============================] - 21s 42ms/step - loss: 0.1822 - accuracy: 0.9605 - val_loss: 0.4871 - val_accuracy: 0.9447
Epoch 62/100
497/497 [==============================] - 22s 44ms/step - loss: 0.0898 - accuracy: 0.9770 - val_loss: 0.4940 - val_accuracy: 0.9606
Epoch 63/100
497/497 [==============================] - 21s 43ms/step - loss: 0.1977 - accuracy: 0.9571 - val_loss: 0.5097 - val_accuracy

In [65]:
y_pred = nn.predict(X_test_df)


In [66]:
y_pred_df = pd.DataFrame((y_pred+0.5).astype(int)).reset_index(drop=True)
y_test_df = y_test_df.reset_index(drop=True)

In [67]:
y_pred_df.shape[1]

43

In [79]:
y_pred_df_tr = y_pred_df.transpose()
y_test_df_tr = y_test_df.transpose()

print(y_pred_df_tr.shape)
print(y_test_df_tr.shape)

mismatches = []

for n in range(y_pred_df.shape[0]):
    row_pred = y_pred_df.iloc[n]
    row_test = y_test_df.iloc[n]
    
    for m in range(len(row_pred)):
        if row_pred[m] != row_test[m]:
            mismatches.append(n)
            break
            
print(f"Accuracy: {(y_pred_df.shape[0] - len(mismatches))/y_pred_df.shape[0]}")


(43, 3921)
(43, 3921)
Accuracy: 0.9520530476919153
